In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('/Volumes/Transcend/data/hist/train_labels.csv')

In [3]:
from collections import defaultdict

In [4]:
import pathlib

In [5]:
root_dir = '/Volumes/Transcend/data/hist/'

In [6]:
train_dir = pathlib.Path(root_dir + 'train/')

In [7]:
from tqdm import tqdm

In [8]:
all_img_labels = []
all_img_path = []
for i in tqdm(train_dir.glob('*.tif')):
    img_id = i.stem
    label = df[df['id'] == img_id]['label'].values[0]
    all_img_path.append(str(i.resolve()))
    all_img_labels.append(label)

7348it [02:11, 55.97it/s]


In [9]:
all_img_path[9]

'/Volumes/Transcend/data/hist/train/0038ee22653a69294588be9707bc5009ecf546ce.tif'

In [10]:
all_img_labels[9]

0

In [11]:
import tensorflow as tf

In [12]:
def _bytes_feature(value):
  """Returns a bytes_list from a string / byte."""
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

In [13]:
def _int64_feature(value):
  """Returns an int64_list from a bool / enum / int / uint."""
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

In [19]:
def get_img_example(img_path, img_label):
    img_str = open(img_path, 'rb').read()
    feature = {
        'img' : _bytes_feature(img_str),
        'label' : _int64_feature(img_label)
    }
    return tf.train.Example(features=tf.train.Features(feature=feature))

In [23]:
idx = 0
base_file = 'hist_rec_'
suffix = '.tfrec'
count = 1
while True:
    tfrec_file_name = base_file + str(count) + suffix
    if idx >= len(all_img_path):
        break
    with tf.compat.v1.python_io.TFRecordWriter(tfrec_file_name) as writer:
        while idx < len(all_img_path):
            path = all_img_path[idx]
            label = all_img_labels[idx]
            example = get_img_example(path, label)
            writer.write(example.SerializeToString())
            idx += 1
            if idx > 0 and idx % 1000 == 0:
                count += 1
                break

In [20]:
with tf.compat.v1.python_io.TFRecordWriter('test.tfrec') as writer:
    count = 0
    while count < 10:
        count += 1
        path = all_img_path[count]
        label = all_img_labels[count]
        example = get_img_example(path, label)
        writer.write(example.SerializeToString())

In [17]:
tf.__version__

'2.0.0-rc0'

In [26]:
def preprocess_img(image):
    '''Resize and normalize a given image.
    
    :param image: Image
    :type image: Raw Tensor Data
    :return: Resized immage
    :rtype: Tensor
    '''
    image = tf.image.decode_jpeg(image)    
    image = tf.image.resize(image, [32, 32])
    image /= 255.0
    return image

In [41]:
def get_file_path_labels(id_label_df, train_dir):
    '''Get list of file paths and corresponding labels
    
    :param id_label_df: Dataframe with id to label mapping
    :type id_label_df: Pandas Dataframe
    :param train_dir: Training directory
    :type train_dir: String
    :return: Paths to images, labels
    :rtype: 2 lists
    '''
    train_dir = pathlib.Path(train_dir)
    img_label = []
    img_path = []
    count = 0
    for i in tqdm(train_dir.glob('*.tif')):
        count += 1
        if count > 1000:
            break
        img_id = i.stem
        label = id_label_df[id_label_df['id'] == img_id]['label'].values[0]
        img_label.append(label)
        img_path.append(str(i.resolve()))
    return img_path, img_label

In [28]:
def _bytes_feature(value):
  """Returns a bytes_list from a string / byte."""
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _int64_feature(value):
  """Returns an int64_list from a bool / enum / int / uint."""
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))
  
def get_img_example(img_path, img_label):
    img_str = open(img_path, 'rb').read()
    feature = {
        'img' : _bytes_feature(img_str),
        'label' : _int64_feature(img_label)
    }
    return tf.train.Example(features=tf.train.Features(feature=feature))

In [42]:
def build_multiple_tf_records(csv_path, train_dir):
    '''Build multiple tf record files
    
    :param csv_path: Path to labels csv
    :type csv_path: String
    :param train_dir: Training directory
    :type train_dir: String
    '''
    img_label_df = pd.read_csv(csv_path)
    img_path, img_label = get_file_path_labels(img_label_df, train_dir)
    idx = 0
    base_file = 'hist_rec_'
    suffix = '.tfrec'
    count = 1
    while True:
        if idx >= len(img_path):
            break
        tfrec_file_name = base_file + str(count) + suffix
        print(tfrec_file_name)
        with tf.compat.v1.python_io.TFRecordWriter(tfrec_file_name) as writer:
            while idx < len(all_img_path):
                path = img_path[idx]
                label = img_label[idx]
                example = get_img_example(path, label)
                writer.write(example.SerializeToString())
                idx += 1
                if idx > 0 and idx % 100 == 0:
                    count += 1
                    break

In [43]:
csv_path = '/Volumes/Transcend/Data/hist/train_labels.csv'
train_dir = '/Volumes/Transcend/Data/hist/train/'
build_multiple_tf_records(csv_path, train_dir)

996it [00:17, 53.45it/s]

hist_rec_1.tfrec
hist_rec_2.tfrec
hist_rec_3.tfrec
hist_rec_4.tfrec
hist_rec_5.tfrec
hist_rec_6.tfrec
hist_rec_7.tfrec
hist_rec_8.tfrec
hist_rec_9.tfrec
hist_rec_10.tfrec


996it [00:30, 53.45it/s]

In [ ]:
image_ds = tf.data.Dataset.from_tensor_slices(all_img_path).map(tf.io.read_file)

In [ ]:
raw_img_tfrec = tf.data.experimental.TFRecordWriter('raw_img.tfrec')

In [ ]:
raw_img_tfrec.write(image_ds)

In [ ]:
def load_preprocess_img(img):
    img = tf.io.read_file(img)
    img = tf.image.decode_jpeg(img)
    img = tf.image.resize(img, [32, 32])
    img = img / 255.0
    return img

In [ ]:
img_ds = tf.data.Dataset.from_tensor_slices(all_img_path).map(load_preprocess_img)

In [ ]:
img_label = tf.data.Dataset.from_tensor_slices(all_img_labels)

In [ ]:
t_img_dataset = tf.data.Dataset.zip((image_ds, img_label))

In [ ]:
len(t_img_dataset)